Frameworks for ML scaling and production
----

# Hadoop

https://hadoop.apache.org/

## Introduction and Use Case

Hadoop is an open-source distributed data management system. It combines tools to store, analyze, and process large-scale pools of data on clusters of servers, without requiring specialized hardware. The "vanilla" version maintained by the Apache Foundation is quite intricate and not entirely stable, so there are many commercial distributions offered by third parties (such as Cloudera, Hortonworks). The major cloud services ([Google](https://cloud.google.com/dataproc?hl=en), Amazon, Microsoft) can also host Hadoop, either with their own out-of-the-box solutions or provided by commercial distributions.

This [table](https://hadoopecosystemtable.github.io/) summarizes libraries and applications within the Hadoop "ecosystem," including those produced by Apache itself and many others.

Cloud data systems like Hadoop represent an alternative to relational databases in order to provide greater scalability and speed at large scales. It is often said that databases can optimize on 2 of 3 goals (CAP): consistency, availability, and partitioning (i.e. scalability). SQL priortizes C and A, while Hadoop prioritizes A and P. Because it lacks the transaction control of relational databases, it is better suited to "behavioral" rather than "line of business" data (such as customer accounts, supply chains, etc). Behavioral data is collected *in aggregate* as side-effect of user activity. Rather than being tracked and queried on the level of individuals, this data is primarily useful for the general patterns than can be seen in it -- hence it is acceptable to deprioritize consistency in a way that would not be workable for business-critical data.


## The Elements of Hadoop

### Hadoop File System (HDFS)

Developed out of a system published by Google, HDFS promises scalability on "commodity" hardware. By default, it employs 3x data redundancy.

### MapReduce Processing API

Implemented in Java.

### HBase

A wide-column, schema-on-read database format that acts as a relatively accessible front-end to data stored in a Hadoop cluster.

### Hive

The query language used for HBase.